In [3]:
#import each models' h5 
import h5py
import os
# os.environ['KERAS_BACKEND'] = 'tensorflow'
# import keras

import glob
import importlib

import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import sys
sys.path.append('MNIST-1/utils/')
from mnistutil import MNISTUitl

In [4]:
# In[]: Data Load
labs = [0,1,2,3,4,5,6,7,8,9]
sx = 28
sy = 28
mn = MNISTUitl()
X, Y, x, y = mn.getdata2(0,0,sx,sy)
#nm , xt, yt = mn.train2(X, Y, x,y,sx,sy,10,50)
xt, yt, xT, yT = mn.trainData(X, Y, x,y,sx,sy,10,50)# test x,y, and train x,y

x_train shape: (60000, 28, 28, 1)
(60000, 28, 28, 1) 60000 train samples (60000,)
10000 test samples


In [5]:
orignal_model= load_model('remove_irrelavant_edges/MNIST-1/mnist.h5')

In [6]:
from sklearn.metrics import accuracy_score
import numpy as np
orignal_model.evaluate(xt, yt)
orignal_predict = orignal_model.predict(xt)
orignal_predict = np.argmax(orignal_predict, axis=1)
score=accuracy_score(orignal_predict, yt)
print(f'Original model accuracy: {score}')

313/313 [==============================] - 0s 252us/step
Original model accuracy: 0.9491


In [7]:
module0=load_model('remove_irrelavant_edges/MNIST-1/module0.h5')
module1=load_model('remove_irrelavant_edges/MNIST-1/module1.h5')
module2=load_model('remove_irrelavant_edges/MNIST-1/module2.h5')
module3=load_model('remove_irrelavant_edges/MNIST-1/module3.h5')
module4=load_model('remove_irrelavant_edges/MNIST-1/module4.h5')
module5=load_model('remove_irrelavant_edges/MNIST-1/module5.h5')
module6=load_model('remove_irrelavant_edges/MNIST-1/module6.h5')
module7=load_model('remove_irrelavant_edges/MNIST-1/module7.h5')
module8=load_model('remove_irrelavant_edges/MNIST-1/module8.h5')
module9=load_model('remove_irrelavant_edges/MNIST-1/module9.h5')

In [8]:
from tqdm import tqdm

In [10]:
finalPred=[]
# print(len(yt))
for i in tqdm(range(len(yt))): #iterate over each test sample
    predictions = []
    maxPredictions = []
#loop through each module for prediction
    for module_idx, module in enumerate([module0, module1, module2, module3, module4, module5, module6, module7, module8, module9]):
        pred = module.predict(xt[i:i+1],verbose=0)
        max_pred_idx = pred.argmax()
        #adjust prediction based on the module index
        predictions.append(max_pred_idx if max_pred_idx == module_idx else 10)
        maxPredictions.append(pred[0][module_idx])
        # print(f'maxPredictions: {maxPredictions}')
    #case 1: all prediction are incorrect
    if predictions.count(10) == 10:
       
        finalPred.append(maxPredictions.index(max(maxPredictions)))
    #case 2: more than one vote, but not unanimous
    elif predictions.count(10) < 9:
        validPredictions = [maxPredictions[i] for i in range(10) if predictions[i] != 10]
        finalPred.append(maxPredictions.index(max(validPredictions)))
    #case 3: exactly one correct vote
    else:
        finalPred.append(predictions.index(next(filter(lambda x: x != 10, predictions))))


100%|██████████| 10000/10000 [25:00<00:00,  6.67it/s] 


In [11]:
from sklearn.metrics import accuracy_score
decomposed_model_pred = accuracy_score(finalPred,yt)
print(f'Decomposed model accuracy is {decomposed_model_pred}')

Decomposed model accuracy is 0.949
